In [4]:
import pandas as pd
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import math

In [48]:
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
col_type = {'anaemia':np.str, 'diabetes':np.str, 
            'high_blood_pressure':np.str, 'sex':np.str, 
            'smoking':np.str, 'DEATH_EVENT':np.str}

df = df.astype(col_type)
x_train, x_test, y_train, y_test = train_test_split(df.drop("DEATH_EVENT", axis=1), df['DEATH_EVENT'], train_size=0.67, random_state=42)


In [53]:
## Reference
gnb = GaussianNB()    
y_pred = gnb.fit(x_train, y_train).predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

ValueError: operands could not be broadcast together with shapes (99,12) (13,) 

In [10]:
type(df['anaemia'])

pandas.core.series.Series

In [11]:
def get_std_dev(nums: pd.core.series.Series):
    return (((nums - nums.mean())**2).aggregate(np.sum)/len(nums))**0.5
    

get_std_dev(df['age'])

11.874901429842655

In [12]:
def prior_prob_categorical(nums:pd.core.series.Series):
    return nums.value_counts(normalize=True)
    
def prior_prob_continuous(nums:pd.core.series.Series):
    nums.mean
# prior_prob_categorical(df['sex'])["1"]

    

In [51]:
def train_naive_bayes(x_train, y_train):
    prob = dict()
    col_y = y_train.name
    classes = np.unique(y_train)
    
    df = x_train
    df[col_y] = y_train
    
    prob['classes'] = {str(k): x for k, x in enumerate(y_train.value_counts(normalize=True))}
    
    for cls in classes:
        tmp = df[df[col_y] == cls]

        for col in x_train.columns:

            if col != col_y:
                if col not in prob.keys():
                    prob[col] = dict()
                
                prob[col][cls] = dict()

                if x_train.dtypes[col] == object:
                    val_probs = prior_prob_categorical(tmp[col])
                    for val in np.unique(tmp[col]):
                        try:
                            prob[col][cls][val] = val_probs[val]

                        except IndexError:
                            prob[col][cls][val] = 0

                else:
                    for cls_2 in classes:
                        prob[col][cls_2] = {
                            'mean': tmp[col].mean(),
                            'std': get_std_dev(tmp[col])}

    return prob

prob = train_naive_bayes(x_train, y_train)
prob

{'classes': {'0': 0.73, '1': 0.27},
 'age': {'0': {'mean': 64.94444444444444, 'std': 11.72274880333844},
  '1': {'mean': 64.94444444444444, 'std': 11.72274880333844}},
 'anaemia': {'0': {'0': 0.5821917808219178, '1': 0.4178082191780822},
  '1': {'0': 0.5740740740740741, '1': 0.42592592592592593}},
 'creatinine_phosphokinase': {'0': {'mean': 629.3888888888889,
   'std': 1201.0406817155706},
  '1': {'mean': 629.3888888888889, 'std': 1201.0406817155706}},
 'diabetes': {'0': {'0': 0.6095890410958904, '1': 0.3904109589041096},
  '1': {'0': 0.5925925925925926, '1': 0.4074074074074074}},
 'ejection_fraction': {'0': {'mean': 33.75925925925926,
   'std': 12.969958429907374},
  '1': {'mean': 33.75925925925926, 'std': 12.969958429907374}},
 'high_blood_pressure': {'0': {'0': 0.6506849315068494,
   '1': 0.3493150684931507},
  '1': {'0': 0.5925925925925926, '1': 0.4074074074074074}},
 'platelets': {'0': {'mean': 257836.0774074074, 'std': 106130.5799387753},
  '1': {'mean': 257836.0774074074, 'std':

In [57]:
def predict_naive_bayes(x_test, prob):
    classes = list(prob[list(prob.keys())[0]].keys())
    y_pred = list()
    
    for i in range(len(x_test)):
        row = x_test.iloc[i]
        cls_prob = list()
        for cls in classes:
            cls = str(cls)
            tmp_prob = 1
            for col in x_test.columns:
                if x_test.dtypes[col] == object:
#                     print(cls, col, prob[col][row[col]])
                    tmp_prob = tmp_prob * prob[col][cls][row[col]]
                    
                else:
                    std = prob[col][cls]['std']
                    exp = math.exp(-1 * ((row[col] - prob[col][cls]['mean'])**2)/(2*(std**2)))
                    
                    tmp = tmp_prob * (exp / (std * (2*math.pi)**0.5))
                    tmp_prob *= tmp
#                     print(row[col], std, exp, tmp)
#                     tmp_prob = tmp_prob * prob[col][row[col]]

                print(cls, tmp_prob)
            cls_prob += [(cls, tmp_prob * prob['classes'][cls])]
        print(cls_prob)
        y_pred += [max(cls_prob, key=lambda x: x[1])[0]]
        
    return np.array(y_pred)
                    



accuracy_score(y_test, predict_naive_bayes(x_test, prob))

0 0.031009473322954957
0 0.0180534604962409
0 1.0817706872181323e-07
0 6.594355559069437e-08
0 1.1913558686394488e-16
0 7.751973117859428e-17
0 3.3817180308397107e-39
0 2.692437428161066e-78
0 5.90619274445193e-157
0 3.843070621389955e-157
0 1.2108304697529995e-157
0 1.90101e-318
1 0.031009473322954957
1 0.017801734685400066
1 1.0518139762940548e-07
1 6.232971711372177e-08
1 1.0643564592512211e-16
1 6.307297536303532e-17
1 2.2387186347882144e-39
1 1.1799669997323847e-78
1 1.1343735732494708e-157
1 6.722213767404271e-158
1 1.867281602056742e-158
1 4.521e-320
[('0', 1.387737e-318), ('1', 1.221e-320)]
0 0.015100034311912036
0 0.0063089184453879055
0 1.2727132900235619e-08
0 7.75832074055459e-09
0 1.842976308797425e-18
0 1.1991969132585985e-18
0 3.3394614930014434e-42
0 2.2642305985287755e-84
0 2.5368155375751528e-169
0 1.6506676443125994e-169
0 5.2007336738616144e-170
0 0.0
1 0.015100034311912036
1 0.006431496095814386
1 1.322649503050868e-08
1 7.83792298104218e-09
1 1.8809890857440465e-1

0 2.9773979523836167e-09
0 2.614585593012759e-19
0 9.133141455044569e-20
0 3.108989660202865e-44
0 2.4562447690211145e-88
0 2.845554719969702e-177
0 9.939951419072247e-178
0 6.808185903474141e-178
0 0.0
1 0.011972922067422823
1 0.005099577917606017
1 7.925543964128568e-09
1 3.228925318719046e-09
1 3.074999814270651e-19
1 1.2527777021102652e-19
1 5.849610364898317e-44
1 8.695347963642275e-88
1 3.566133422605463e-176
1 1.452869172172596e-176
1 1.0492944021246526e-176
1 0.0
[('0', 0.0), ('1', 0.0)]
0 0.02355624600100397
0 0.013714252808803682
0 5.7079825334624203e-08
0 3.479523599165448e-08
0 1.7003301006700166e-17
0 5.939509255765127e-18
0 1.3203999248100667e-40
0 3.856880584763902e-81
0 3.4212821808408705e-164
0 2.226176761506046e-164
0 1.5247786037712641e-164
0 0.0
1 0.02355624600100397
1 0.013523030111687465
1 5.549915408206571e-08
1 3.288838760418708e-08
1 1.5190736648438978e-17
1 6.188818634549213e-18
1 1.4335732067357258e-40
1 4.546371931164743e-81
1 4.753860503573058e-164
1 2.8171

0 0.011863064235401705
0 4.5179420555410646e-08
0 2.7540879653640737e-08
0 1.8573202992645372e-17
0 6.48789967551311e-18
0 1.189567888635636e-40
0 3.6492784724956384e-81
0 7.963721883660971e-163
0 5.1818738284095366e-163
0 1.6326451788139633e-163
0 0.0
1 0.028393563579813916
1 0.01209355485806889
1 4.69520815207593e-08
1 2.782345571600551e-08
1 1.895628931837488e-17
1 7.72293268526384e-18
1 1.6855649036118115e-40
1 7.326887116059549e-81
1 3.2102664159330547e-162
1 1.902380098330699e-162
1 5.2843891620297197e-163
1 0.0
[('0', 0.0), ('1', 0.0)]
0 0.015100034311912036
0 0.00879111586652413
0 2.565088167455802e-08
0 1.5636496363257972e-08
0 3.4337807907880835e-18
0 2.2343094186634792e-18
0 1.152056604775955e-41
0 2.2935803135980323e-83
0 4.245017622139731e-167
0 1.4828486214323716e-167
0 1.0156497407071039e-167
0 0.0
1 0.015100034311912036
1 0.00866853821609765
1 2.494054994126943e-08
1 1.4779585150381883e-08
1 3.0677372400086273e-18
1 1.817918364449557e-18
1 7.626687281205264e-42
1 1.0051

0 7.023189468991575e-172
0 4.569883558590409e-172
0 1.4398263266791698e-172
0 0.0
1 0.018497566510368973
1 0.007878593143305303
1 1.9289415183401987e-08
1 7.858650630274884e-09
1 1.821482725544667e-18
1 7.420855548515309e-19
1 1.6000884938577814e-42
1 5.437828640187572e-85
1 8.801668976138427e-171
1 5.21580383771166e-171
1 1.4488343993643502e-171
1 0.0
[('0', 0.0), ('1', 0.0)]
0 0.007876051649148912
0 0.0032906791136855045
0 3.5000146522961178e-09
0 2.1335705757147567e-09
0 6.393044053040417e-20
0 4.159857431772874e-20
0 6.355839846543587e-45
0 9.56974519527669e-90
0 7.390134483279549e-180
0 4.808649150079159e-180
0 3.2935953082733965e-180
0 0.0
1 0.007876051649148912
1 0.0033546145913041663
1 3.637341321739882e-09
1 2.155461523994004e-09
1 6.524905410366665e-20
1 3.8666106135506163e-20
1 5.491322241015882e-45
1 7.143455488197079e-90
1 4.117828724107497e-180
1 2.4401947994711095e-180
1 1.7623629107291346e-180
1 0.0
[('0', 0.0), ('1', 0.0)]
0 0.02355624600100397
0 0.013714252808803682
0

0.5757575757575758

In [149]:
y_pred

array(['0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1',
       '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0',
       '0', '0', '1', '1', '0', '0', '0', '0'], dtype='<U1')

In [26]:
{k: x for k, x in enumerate(y_train.value_counts(normalize=True))}

{0: 0.73, 1: 0.27}

In [23]:
y_train.value_counts(normalize=True).index


Index(['0', '1'], dtype='object')